In [14]:
#importando a biblioteca PySpark
import time
import pyspark

#função para ver o tempo de execução
tempo_inicial = time.time() # em segundos 
tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

0.0 segundos


In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Read CSV File into DataFrame').getOrCreate()

In [16]:
#abrindo o arquivo csv do pokemon
df = spark.read.csv('Pokemon_Data.csv', sep=',', inferSchema=True, header=True)

#df = spark.read.format("csv").load("Pokemon_Data.csv")

In [17]:
#verificando os campos e os tipos de dados
df.printSchema()

root
 |-- Pokemon: string (nullable = true)
 |-- Pokemon Number: integer (nullable = true)
 |-- Stat Total: integer (nullable = true)
 |-- HP: integer (nullable = true)
 |-- Attack: integer (nullable = true)
 |-- Defense: integer (nullable = true)
 |-- Sp.Atk: integer (nullable = true)
 |-- Sp.Def: integer (nullable = true)
 |-- Speed: integer (nullable = true)
 |-- Mega: boolean (nullable = true)
 |-- First Type: string (nullable = true)
 |-- Second Type: string (nullable = true)
 |-- Species: string (nullable = true)
 |-- First Ability: string (nullable = true)
 |-- Second Ability: string (nullable = true)
 |-- Hidden Ability: string (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Egg Group 1: string (nullable = true)
 |-- Egg Group 2: string (nullable = true)
 |-- Is Sub Legendary: boolean (nullable = true)
 |-- Is Legendary: boolean (nullable = true)
 |-- Is Mythical: boolean (nullable = true)



In [18]:
#testando a tabela
df.select('Pokemon', 'Pokemon Number').show()

+--------------------+--------------+
|             Pokemon|Pokemon Number|
+--------------------+--------------+
|           Bulbasaur|             1|
|             Ivysaur|             2|
|            Venusaur|             3|
|Venusaur(Mega Ven...|             3|
|          Charmander|             4|
|          Charmeleon|             5|
|           Charizard|             6|
|Charizard(Mega Ch...|             6|
|Charizard(Mega Ch...|             6|
|            Squirtle|             7|
|           Wartortle|             8|
|           Blastoise|             9|
|Blastoise(Mega Bl...|             9|
|            Caterpie|            10|
|             Metapod|            11|
|          Butterfree|            12|
|              Weedle|            13|
|              Kakuna|            14|
|            Beedrill|            15|
|Beedrill(Mega Bee...|            15|
+--------------------+--------------+
only showing top 20 rows



In [19]:
#separando a entidade pokemon
tempo_inicial = time.time()
pokemon = df.select('Pokemon', 'Pokemon Number')

pokemon.show()
tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+--------------+
|             Pokemon|Pokemon Number|
+--------------------+--------------+
|           Bulbasaur|             1|
|             Ivysaur|             2|
|            Venusaur|             3|
|Venusaur(Mega Ven...|             3|
|          Charmander|             4|
|          Charmeleon|             5|
|           Charizard|             6|
|Charizard(Mega Ch...|             6|
|Charizard(Mega Ch...|             6|
|            Squirtle|             7|
|           Wartortle|             8|
|           Blastoise|             9|
|Blastoise(Mega Bl...|             9|
|            Caterpie|            10|
|             Metapod|            11|
|          Butterfree|            12|
|              Weedle|            13|
|              Kakuna|            14|
|            Beedrill|            15|
|Beedrill(Mega Bee...|            15|
+--------------------+--------------+
only showing top 20 rows

0.09515118598937988 segundos


In [20]:
#separando o tipos de pokemon
tempo_inicial = time.time()
from pyspark.sql.functions import *
type_pokemon = df.select('First Type').distinct().withColumnRenamed('First Type', 'Type').orderBy('Type')

#criando uma coluna para ID dos tipos
type_pokemon = type_pokemon.withColumn("ID", monotonically_increasing_id())

type_pokemon.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------+---+
|    Type| ID|
+--------+---+
|     Bug|  0|
|    Dark|  1|
|  Dragon|  2|
|Electric|  3|
|   Fairy|  4|
|Fighting|  5|
|    Fire|  6|
|  Flying|  7|
|   Ghost|  8|
|   Grass|  9|
|  Ground| 10|
|     Ice| 11|
|  Normal| 12|
|  Poison| 13|
| Psychic| 14|
|    Rock| 15|
|   Steel| 16|
|   Water| 17|
+--------+---+

1.1033785343170166 segundos


In [21]:
#fazendo a tabela de stats
tempo_inicial = time.time()
sp = df.withColumnRenamed('Sp.Atk', 'Sp_Atk')
sp = sp.withColumnRenamed('Sp.Def', 'Sp_Def')
stats = sp.select('Pokemon', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Stat Total')

stats.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+---+------+-------+------+------+-----+----------+
|             Pokemon| HP|Attack|Defense|Sp_Atk|Sp_Def|Speed|Stat Total|
+--------------------+---+------+-------+------+------+-----+----------+
|           Bulbasaur| 45|    49|     49|    65|    65|   45|       318|
|             Ivysaur| 60|    62|     63|    80|    80|   60|       405|
|            Venusaur| 80|    82|     83|   100|   100|   80|       525|
|Venusaur(Mega Ven...| 80|   100|    123|   122|   120|   80|       625|
|          Charmander| 39|    52|     43|    60|    50|   65|       309|
|          Charmeleon| 58|    64|     58|    80|    65|   80|       405|
|           Charizard| 78|    84|     78|   109|    85|  100|       534|
|Charizard(Mega Ch...| 78|   130|    111|   130|    85|  100|       634|
|Charizard(Mega Ch...| 78|   104|     78|   159|   115|  100|       634|
|            Squirtle| 44|    48|     65|    50|    64|   43|       314|
|           Wartortle| 59|    63|     80|    65|   

In [22]:
#separando egg group
tempo_inicial = time.time()
egg_group = df.select('Egg Group 1').distinct().withColumnRenamed('Egg Group 1', 'Egg_Group').orderBy('Egg_Group')

#criando uma coluna para ID dos eggs groups
egg_group = egg_group.withColumn("ID", monotonically_increasing_id())

egg_group.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+------------+---+
|   Egg_Group| ID|
+------------+---+
|   Amorphous|  0|
|         Bug|  1|
|       Ditto|  2|
|      Dragon|  3|
|       Fairy|  4|
|       Field|  5|
|      Flying|  6|
|       Grass|  7|
|  Human-Like|  8|
|     Mineral|  9|
|     Monster| 10|
|Undiscovered| 11|
|     Water 1| 12|
|     Water 2| 13|
|     Water 3| 14|
+------------+---+

0.3928201198577881 segundos


In [23]:
#separando as habilidades
tempo_inicial = time.time()
ability1 = df.select('First Ability').distinct().withColumnRenamed('First Ability', 'Ability').orderBy('Ability')
ability2 = df.select('Second Ability').distinct().withColumnRenamed('Second Ability', 'Ability').orderBy('Ability')
ability3 = df.select('Hidden Ability').distinct().withColumnRenamed('Hidden Ability', 'Ability').orderBy('Ability')

ability = ability1.union(ability2).distinct().orderBy('Ability')
ability = ability.union(ability3).distinct().orderBy('Ability')

#criando uma coluna para ID das habilidades
ability = ability.withColumn("ID", monotonically_increasing_id())

ability.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+-------------+---+
|      Ability| ID|
+-------------+---+
| Adaptability|  0|
|     Aerilate|  1|
|    Aftermath|  2|
|     Air Lock|  3|
|     Analytic|  4|
|  Anger Point|  5|
|  Anger Shell|  6|
| Anticipation|  7|
|   Arena Trap|  8|
|   Armor Tail|  9|
|   Aroma Veil| 10|
|       As One| 11|
|   Aura Break| 12|
|   Bad Dreams| 13|
|   Ball Fetch| 14|
|      Battery| 15|
| Battle Armor| 16|
|  Battle Bond| 17|
|Beads of Ruin| 18|
|  Beast Boost| 19|
+-------------+---+
only showing top 20 rows

2.6447181701660156 segundos


In [25]:
#criando uma nova tabela para categorias
tempo_inicial = time.time()
category = spark.read.csv('category.csv', sep=',', inferSchema=True, header=True)

#criando uma coluna para ID das habilidades
category = category.withColumn("ID", monotonically_increasing_id())

category.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+-------------+---+
|     Category| ID|
+-------------+---+
|Sub Legendary|  0|
|    Legendary|  1|
|     Mythical|  2|
+-------------+---+

0.17174267768859863 segundos


In [26]:
#criando a tabela de pokemon egg group 1 e egg group 2
tempo_inicial = time.time()
pokemon_egg_group1 = df.select('Pokemon', 'Egg Group 1').withColumnRenamed('Egg Group 1', 'Egg_Group_1')
pokemon_egg_group2 = df.select('Pokemon', 'Egg Group 2').withColumnRenamed('Egg Group 2', 'Egg_Group_2')

pokemon_egg_group1 = pokemon_egg_group1.join(egg_group, pokemon_egg_group1.Egg_Group_1 == egg_group.Egg_Group, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Egg Group 1 ID')
pokemon_egg_group2 = pokemon_egg_group2.join(egg_group, pokemon_egg_group2.Egg_Group_2 == egg_group.Egg_Group, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Egg Group 2 ID')

pokemon_egg_group1.show()
pokemon_egg_group2.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+--------------+
|             Pokemon|Egg Group 1 ID|
+--------------------+--------------+
|           Bulbasaur|             7|
|             Ivysaur|             7|
|            Venusaur|             7|
|Venusaur(Mega Ven...|             7|
|          Charmander|             3|
|          Charmeleon|             3|
|           Charizard|             3|
|Charizard(Mega Ch...|             3|
|Charizard(Mega Ch...|             3|
|            Squirtle|            10|
|           Wartortle|            10|
|           Blastoise|            10|
|Blastoise(Mega Bl...|            10|
|            Caterpie|             1|
|             Metapod|             1|
|          Butterfree|             1|
|              Weedle|             1|
|              Kakuna|             1|
|            Beedrill|             1|
|Beedrill(Mega Bee...|             1|
+--------------------+--------------+
only showing top 20 rows

+--------------------+--------------+
|             Pokemon|Eg

In [27]:
#criando a tabela de pokemon first ability, second ability e hidden ability
tempo_inicial = time.time()
pokemon_first_ability = df.select('Pokemon', 'First Ability').withColumnRenamed('First Ability', 'First_Ability')
pokemon_second_ability = df.select('Pokemon', 'Second Ability').withColumnRenamed('Second Ability', 'Second_Ability')
pokemon_hidden_ability = df.select('Pokemon', 'Hidden Ability').withColumnRenamed('Hidden Ability', 'Hidden_Ability')

pokemon_first_ability = pokemon_first_ability.join(ability, pokemon_first_ability.First_Ability == ability.Ability, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Ability ID')
pokemon_second_ability = pokemon_second_ability.join(ability, pokemon_second_ability.Second_Ability == ability.Ability, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Ability ID')
pokemon_hidden_ability = pokemon_hidden_ability.join(ability, pokemon_hidden_ability.Hidden_Ability == ability.Ability, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Ability ID')

pokemon_first_ability.show()
pokemon_second_ability.show()
pokemon_hidden_ability.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+----------+
|             Pokemon|Ability ID|
+--------------------+----------+
|           Porygon-Z|         0|
|Lucario(Mega Luca...|         0|
|Beedrill(Mega Bee...|         0|
|Salamence(Mega Sa...|         1|
| Pinsir(Mega Pinsir)|         1|
|            Drifblim|         2|
|            Drifloon|         2|
|            Rayquaza|         3|
|               Klawf|         6|
|             Flittle|         7|
|           Toxicroak|         7|
|            Croagunk|         7|
|Wormadam(Trash Cl...|         7|
|Wormadam(Sandy Cl...|         7|
|            Wormadam|         7|
|  Oinkologne(Female)|        10|
|             Lechonk|        10|
|Calyrex(Shadow Ri...|        11|
|  Calyrex(Ice Rider)|        11|
|  Zygarde(10% Forme)|        12|
+--------------------+----------+
only showing top 20 rows

+--------------------+----------+
|             Pokemon|Ability ID|
+--------------------+----------+
| Basculegion(Female)|         0|
|         Basculegion|

In [28]:
#criando a tabela de pokemon type
tempo_inicial = time.time()
pokemon_first_type = df.select('Pokemon', 'First Type').withColumnRenamed('First Type', 'First_Type')
pokemon_second_type = df.select('Pokemon', 'Second Type').withColumnRenamed('Second Type', 'Second_Type')

pokemon_first_type = pokemon_first_type.join(type_pokemon, pokemon_first_type.First_Type == type_pokemon.Type, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Type ID')
pokemon_second_type = pokemon_second_type.join(type_pokemon, pokemon_second_type.Second_Type == type_pokemon.Type, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Type ID')

pokemon_first_type.show()
pokemon_second_type.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+-------+
|             Pokemon|Type ID|
+--------------------+-------+
|           Bulbasaur|      9|
|             Ivysaur|      9|
|            Venusaur|      9|
|Venusaur(Mega Ven...|      9|
|          Charmander|      6|
|          Charmeleon|      6|
|           Charizard|      6|
|Charizard(Mega Ch...|      6|
|Charizard(Mega Ch...|      6|
|            Squirtle|     17|
|           Wartortle|     17|
|           Blastoise|     17|
|Blastoise(Mega Bl...|     17|
|            Caterpie|      0|
|             Metapod|      0|
|          Butterfree|      0|
|              Weedle|      0|
|              Kakuna|      0|
|            Beedrill|      0|
|Beedrill(Mega Bee...|      0|
+--------------------+-------+
only showing top 20 rows

+--------------------+-------+
|             Pokemon|Type ID|
+--------------------+-------+
|           Bulbasaur|     13|
|             Ivysaur|     13|
|            Venusaur|     13|
|Venusaur(Mega Ven...|     13|
|           C

In [29]:
#criando a tabela de pokemon category
tempo_inicial = time.time()
pokemon_category_sub = df.select('Pokemon', 'Is Sub Legendary').withColumnRenamed('Is Sub Legendary', 'Is_Sub_Legendary').filter(('Is_Sub_Legendary == true'))
pokemon_category_leg = df.select('Pokemon', 'Is Legendary').withColumnRenamed('Is Legendary', 'Is_Legendary').filter(('Is_Legendary == true'))
pokemon_category_my = df.select('Pokemon', 'Is Mythical').withColumnRenamed('Is Mythical', 'Is_Mythical').filter(('Is_Mythical == true'))

#colocando uma coluna padrão para cada categoria
from pyspark.sql.functions import lit
pokemon_category_sub = pokemon_category_sub.withColumn("Category ID", lit(0)).select('Pokemon', 'Category ID')
pokemon_category_leg = pokemon_category_leg.withColumn("Category ID", lit(1)).select('Pokemon', 'Category ID')
pokemon_category_my = pokemon_category_my.withColumn("Category ID", lit(2)).select('Pokemon', 'Category ID')

#unindo as tabelas com pokemon e category ID
pokemon_category = pokemon_category_sub.union(pokemon_category_leg)
pokemon_category = pokemon_category.union(pokemon_category_my)

pokemon_category.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+-----------+
|             Pokemon|Category ID|
+--------------------+-----------+
|            Articuno|          0|
|Articuno(Galarian...|          0|
|              Zapdos|          0|
|Zapdos(Galarian Z...|          0|
|             Moltres|          0|
|Moltres(Galarian ...|          0|
|              Raikou|          0|
|               Entei|          0|
|             Suicune|          0|
|            Regirock|          0|
|              Regice|          0|
|           Registeel|          0|
|              Latias|          0|
| Latias(Mega Latias)|          0|
|              Latios|          0|
| Latios(Mega Latios)|          0|
|                Uxie|          0|
|             Mesprit|          0|
|               Azelf|          0|
|             Heatran|          0|
+--------------------+-----------+
only showing top 20 rows

0.27287888526916504 segundos


In [30]:
#criando a tabela de pokemon mega
tempo_inicial = time.time()
pokemon_mega = df.select('Pokemon').filter(('Mega == True'))

pokemon_mega.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+
|             Pokemon|
+--------------------+
|Venusaur(Mega Ven...|
|Charizard(Mega Ch...|
|Charizard(Mega Ch...|
|Blastoise(Mega Bl...|
|Beedrill(Mega Bee...|
|Pidgeot(Mega Pidg...|
|Alakazam(Mega Ala...|
|Slowbro(Mega Slow...|
| Gengar(Mega Gengar)|
|Kangaskhan(Mega K...|
| Pinsir(Mega Pinsir)|
|Gyarados(Mega Gya...|
|Aerodactyl(Mega A...|
|Mewtwo(Mega Mewtw...|
|Mewtwo(Mega Mewtw...|
|Ampharos(Mega Amp...|
|Steelix(Mega Stee...|
| Scizor(Mega Scizor)|
|Heracross(Mega He...|
|Houndoom(Mega Hou...|
+--------------------+
only showing top 20 rows

0.07895612716674805 segundos


In [31]:
#separando species
tempo_inicial = time.time()
species = df.select('Species').distinct()

#criando uma coluna para ID das species
species = species.withColumn("ID", monotonically_increasing_id())

species.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+-------------------+---+
|            Species| ID|
+-------------------+---+
|  Long Nose Pokémon|  0|
| Cat Ferret Pokémon|  1|
| Artificial Pokémon|  2|
|  Moonlight Pokémon|  3|
|    Firefly Pokémon|  4|
|     Family Pokémon|  5|
|      Olive Pokémon|  6|
|       Bulb Pokémon|  7|
|    Balloon Pokémon|  8|
|   Sleeping Pokémon|  9|
|    Victory Pokémon| 10|
|  Centipede Pokémon| 11|
|   Shedding Pokémon| 12|
|    Glowing Pokémon| 13|
|Frigatebird Pokémon| 14|
|      Frill Pokémon| 15|
|     Flower Pokémon| 16|
|    Licking Pokémon| 17|
|     Symbol Pokémon| 18|
|    Whisper Pokémon| 19|
+-------------------+---+
only showing top 20 rows

0.33898162841796875 segundos


In [32]:
#separando generation
tempo_inicial = time.time()
generation = df.select('Generation').distinct()

#criando uma coluna para ID das generations
generation = generation.withColumn("ID", monotonically_increasing_id())

generation.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+------------+---+
|  Generation| ID|
+------------+---+
|Generation 9|  0|
|Generation 8|  1|
|Generation 5|  2|
|Generation 1|  3|
|Generation 3|  4|
|Generation 2|  5|
|Generation 4|  6|
|Generation 7|  7|
|Generation 6|  8|
+------------+---+

0.12592816352844238 segundos


In [33]:
#criando a tabela de pokemon species
tempo_inicial = time.time()
pokemon_species = df.select('Pokemon', 'Species')

pokemon_species = pokemon_species.join(species, pokemon_species.Species == species.Species, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Specie ID')

pokemon_species.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+---------+
|             Pokemon|Specie ID|
+--------------------+---------+
|           Bulbasaur|      257|
|             Ivysaur|      257|
|            Venusaur|      257|
|Venusaur(Mega Ven...|      257|
|          Charmander|       96|
|          Charmeleon|      231|
|           Charizard|      231|
|Charizard(Mega Ch...|      231|
|Charizard(Mega Ch...|      231|
|            Squirtle|       72|
|           Wartortle|      551|
|           Blastoise|      232|
|Blastoise(Mega Bl...|      232|
|            Caterpie|      589|
|             Metapod|       62|
|          Butterfree|      597|
|              Weedle|       95|
|              Kakuna|       62|
|            Beedrill|      303|
|Beedrill(Mega Bee...|      303|
+--------------------+---------+
only showing top 20 rows

0.4096686840057373 segundos


In [34]:
#criando a tabela de pokemon generation
tempo_inicial = time.time()
pokemon_generation = df.select('Pokemon', 'Generation')

pokemon_generation = pokemon_generation.join(generation, pokemon_generation.Generation == generation.Generation, 'inner').select('Pokemon', 'ID').withColumnRenamed('ID', 'Generation ID')

pokemon_generation.show()

tempo_final = time.time()
print(f"{tempo_final - tempo_inicial} segundos")

+--------------------+-------------+
|             Pokemon|Generation ID|
+--------------------+-------------+
|           Bulbasaur|            3|
|             Ivysaur|            3|
|            Venusaur|            3|
|Venusaur(Mega Ven...|            8|
|          Charmander|            3|
|          Charmeleon|            3|
|           Charizard|            3|
|Charizard(Mega Ch...|            8|
|Charizard(Mega Ch...|            8|
|            Squirtle|            3|
|           Wartortle|            3|
|           Blastoise|            3|
|Blastoise(Mega Bl...|            8|
|            Caterpie|            3|
|             Metapod|            3|
|          Butterfree|            3|
|              Weedle|            3|
|              Kakuna|            3|
|            Beedrill|            3|
|Beedrill(Mega Bee...|            8|
+--------------------+-------------+
only showing top 20 rows

0.17653894424438477 segundos
